In [ ]:
## Imports

import pandas as pd
import numpy as np
import os

import random

from typing import Union, List

from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

from scipy.stats import spearmanr

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import RobustScaler

from sklearn.utils import shuffle

In [ ]:
# Define the objective function for each ML algorithm

CV = 5
njobs = -1

def objective_xgboost(space, X_train, y_train, seed):
    model = XGBRegressor(n_estimators=space['n_estimators'], max_depth=space['max_depth'], 
                         learning_rate=space['learning_rate'], gamma=space['gamma'],
                         min_child_weight=space['min_child_weight'], subsample=space['subsample'],
                         colsample_bytree=space['colsample_bytree'], reg_alpha=space['reg_alpha'],
                         reg_lambda=space['reg_lambda'], random_state=seed, n_jobs=njobs)
    
    rmse = -cross_val_score(model, X_train, y_train, cv=CV, scoring='neg_root_mean_squared_error').mean()

    return {'loss': rmse, 'status': STATUS_OK}

def objective_svm(space, X_train, y_train):
    model = SVR(kernel=space['kernel'], gamma=space['gamma'], C=space['C'], epsilon=space['epsilon'])
    
    rmse = -cross_val_score(model, X_train, y_train, cv=CV, scoring='neg_root_mean_squared_error').mean()

    return {'loss': rmse, 'status': STATUS_OK}

def objective_rf(space, X_train, y_train, seed):
    model = RandomForestRegressor(n_estimators=space['n_estimators'], max_depth=space['max_depth'], 
                                  max_features=space['max_features'], min_samples_split=space['min_samples_split'],
                                  random_state=seed, n_jobs=njobs)
    
    rmse = -cross_val_score(model, X_train, y_train, cv=CV, scoring='neg_root_mean_squared_error').mean()

    return {'loss': rmse, 'status': STATUS_OK}

def objective_knn(space, X_train, y_train):
    model = KNeighborsRegressor(n_neighbors=space['n_neighbors'], weights=space['weights'], 
                                metric=space['metric'], n_jobs=njobs)
    
    rmse = -cross_val_score(model, X_train, y_train, cv=CV, scoring='neg_root_mean_squared_error').mean()

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# Define the parameter search space for each ML algorithm
xgboost_space = {
    'n_estimators': hp.choice('n_estimators', [50, 100, 200, 500]), 
    'max_depth': hp.choice('max_depth', [3, 5, 8, 12, 15, 20]), 
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-2, 0.05, 0.1, 0.2]),
    'gamma': hp.choice ('gamma', [0, 0.1, 0.25, 0.5, 1]),  
    'min_child_weight': hp.choice('min_child_weight', [1, 3, 5]), 
    'subsample': hp.choice('subsample', [0.25, 0.5, 0.7, 1.0]), 
    'colsample_bytree': hp.choice('colsample_bytree', [0.25, 0.5, 0.7, 1.0]), 
    'reg_alpha': hp.choice('reg_alpha', [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1]), 
    'reg_lambda': hp.choice('reg_lambda', [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1]) 
}

svm_space = {
    'kernel': hp.choice('weights', ['linear', 'rbf']),
    'gamma': hp.choice('gamma', ['scale', 'auto', 0.001, 0.01, 0.1]),
    'C': hp.choice('C', [0.1, 1, 10, 100, 1000]),
    'epsilon': hp.choice('epsilon', [0.01, 0.05, 0.1, 0.2])
}

rf_space = {
    'n_estimators': hp.randint('n_estimators', 10, 250),
    'max_depth': hp.randint('max_depth', 1, 50),
    'max_features': hp.uniform('max_features', 0.1, 0.999),
    'min_samples_split': hp.randint('min_samples_split', 2, 25)
}

knn_space = {
    'n_neighbors': hp.randint('n_neighbors', 5, 100),
    'weights': hp.choice('weights', ['minkowski', 'euclidean', 'manhattan']),
    'metric': hp.choice('metric', ['uniform', 'distance'])
}  'metric': hp.choice('metric', metric_options)

In [ ]:
def train_and_evaluate(endpoint: str, feature: str, sources_list: Union[str, List[str]], model_type: str, save_results: bool, experiment_name: str):
     
    cv_sets_dir = os.path.join(os.getcwd(), '..', 'data', endpoint, 'cv_sets_scaling', feature)
    cv_sets_files = os.listdir(cv_sets_dir)

    # Initialize metrics variable
    metrics = []

    # Iterate over seeds:
    for seed in range(1,6):
        random.seed(seed)
        print(f'### Seed {seed} ###')

        # Iterate over folds
        for i in range(1,6):
            print(f'Fold {i}')

            # Select test data
            test_files = [file for file in cv_sets_files if f'fold{i}' in file]

            if len(sources_list) == 1:
                # Select and load training data
                training_files = [file for file in cv_sets_files if (sources_list[0] in file) and (f'fold{i}' not in file)]
                training_data = [pd.read_csv(os.path.join(cv_sets_dir, file), sep="\t") for file in training_files]
                training_df = pd.concat(training_data, ignore_index=True)
                # Shuffle data and Set input features and target variable
                training_df = training_df.sample(frac=1).reset_index(drop=True)
                X_train, y_train = training_df.iloc[:,5:], training_df['value']

            else:
                X_train_data = []
                y_train_data = []
                transformers_dict = {}

                for source in sources_list:
                    # Select and load training data
                    source_files = [file for file in cv_sets_files if (source in file) and (f'fold{i}' not in file)]
                    source_data = [pd.read_csv(os.path.join(cv_sets_dir, file), sep="\t") for file in source_files]
                    source_df = pd.concat(source_data, ignore_index=True)
                    # Set input features and target variable
                    X_train, y_train = source_df.iloc[:,5:], source_df['value']
                    # Scale target value
                    transformer = RobustScaler()
                    y_train = pd.Series(transformer.fit_transform(y_train.values.reshape(-1, 1)).flatten(), index=y_train.index)

                    X_train_data.append(X_train)
                    y_train_data.append(y_train)
                    transformers_dict[source] = transformer

                # Concatenate features and target variable and Shuffle data
                X_train = pd.concat(X_train_data, axis=0)
                y_train = pd.concat(y_train_data, axis=0)
                X_train, y_train = shuffle(X_train, y_train, random_state=seed)

            # Train the ML model
            if model_type == 'XGBoost':
                # Perform Hyperparameter Tuning
                trials=Trials()
                best = fmin(fn=lambda space: objective_xgboost(space, X_train, y_train, seed), return_argmin=False, 
                            space=xgboost_space, algo=tpe.suggest, max_evals=50, timeout=600, trials=trials, rstate=np.random.default_rng(seed))
                print(f'Best hyperparameters: {best}')
                # Train the final model with the best hyperparameters
                model = XGBRegressor(n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']),
                                     learning_rate=float(best['learning_rate']), gamma=float(best['gamma']),
                                     min_child_weight=int(best['min_child_weight']), subsample=float(best['subsample']),
                                     colsample_bytree=float(best['colsample_bytree']), reg_alpha=float(best['reg_alpha']),
                                     reg_lambda=float(best['reg_lambda']), random_state=seed)
            
            elif model_type == 'SVM':
                # Perform Hyperparameter Tuning
                trials=Trials()
                best = fmin(fn=lambda space: objective_svm(space, X_train[:1000], y_train[:1000]), return_argmin=False, 
                            space=svm_space, algo=tpe.suggest, max_evals=50, timeout=600, trials=trials, rstate=np.random.default_rng(seed))
                print(f'Best hyperparameters: {best}')
                # Train the final model with the best hyperparameters
                model = SVR(kernel=str(best['kernel']), gamma=float(best['gamma']), 
                            C=float(best['C']), epsilon=float(best['epsilon']),)
                
            elif model_type == 'RF':
                # Perform Hyperparameter Tuning
                trials=Trials()
                best = fmin(fn=lambda space: objective_rf(space, X_train, y_train, seed), return_argmin=False, 
                            space=rf_space, algo=tpe.suggest, max_evals=50, timeout=600, trials=trials, rstate=np.random.default_rng(seed))
                print(f'Best hyperparameters: {best}')
                # Train the final model with the best hyperparameters
                model = RandomForestRegressor(n_estimators=int(best['n_estimators']), max_depth=int(best['max_depth']),
                                              max_features=float(best['max_features']), min_samples_split=int(best['min_samples_split']),
                                              random_state=seed)

            elif model_type == 'KNN':
                # Perform Hyperparameter Tuning
                trials=Trials()
                best = fmin(fn=lambda space: objective_knn(space, X_train, y_train), return_argmin=False, 
                            space=knn_space, algo=tpe.suggest, max_evals=50, timeout=600, trials=trials, rstate=np.random.default_rng(seed))
                print(f'Best hyperparameters: {best}')
                # Train the final model with the best hyperparameters
                model = KNeighborsRegressor(n_neighbors=int(best['n_neighbors']), weights=str(best['weights']), 
                                            metric=str(best['metric']))
                    
            model.fit(X_train, y_train)

            # Evaluate model performance on each test set
            for file in test_files:
                # Load test data
                test_data = pd.read_csv(os.path.join(cv_sets_dir, file), sep='\t')
                # Set input features and target variable
                X_test, y_test = test_data.iloc[:,5:], test_data['value']
                # Predict 
                y_pred = model.predict(X_test)

                if len(sources_list) > 1:
                    # Perform inverse transformation of the predictions
                    transformer = transformers_dict[file[:-10]]
                    y_pred = transformer.inverse_transform(y_pred.reshape(-1, 1)).flatten()

                # Compute metrics
                rmse = root_mean_squared_error(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)
                spearman_corr, spearman_pvalue = spearmanr(y_test, y_pred)  

                metrics.append(['_'.join(file.split('.')[0].split('_')[:-1]), seed, i, test_data.shape[0], rmse, mae, r2, spearman_corr, spearman_pvalue])

        metrics_df = pd.DataFrame(data=metrics, columns=['ref','seed','fold','n_mols','rmse','mae','r2','spearman_corr','spearman_pvalue'])

    # Group by source calculate mean and std for each metric 
    metrics_summary = metrics_df.groupby('ref')[['rmse','mae','r2','spearman_corr','spearman_pvalue']].agg(['mean', 'std']).reset_index()

    # Save results
    if save_results:
        results_dir = os.path.join(os.getcwd(), '..', 'results_scaling', endpoint, experiment_name)
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
        metrics_df.to_csv(os.path.join(results_dir, f'{model_type}_{feature}_metrics_folds.tsv'), sep='\t', index=False)
        metrics_summary.to_csv(os.path.join(results_dir, f'{model_type}_{feature}_metrics_cv.tsv'), sep='\t', index=False)
    
    return metrics_df, metrics_summary

# Train and Evaluate ML models

## Half-life

In [ ]:
endpoint = 'half_life'
features = ['ecfp4', 'rdkit_ecfp4']
model_types = ['XGBoost', 'SVM', 'RF', 'KNN']
cases_list = [['Homogenous'],
              ['Fan'],
              ['Homogenous', 'Fan']]

In [ ]:
for feature in features:
    for model in model_types:
        for case in cases_list:

            print(f'\n### {", ".join(case)} --> Features: {feature.upper()}, Model: {model} ###\n')
            
            metrics_df, metrics_summary = train_and_evaluate(endpoint, feature=feature, sources_list=case, model_type=model, 
                                                             save_results=True, experiment_name=f'{"_".join(case)}')
            print(metrics_summary)

## Clearance

In [ ]:
endpoint = 'clearance'
features = ['ecfp4', 'rdkit_ecfp4']
model_types = ['XGBoost', 'SVM', 'RF', 'KNN']
cases_list = [['Homogenous'],
              ['Astrazeneca'],
              ['Homogenous', 'Astrazeneca']]

In [ ]:
for feature in features:
    for model in model_types:
        for case in cases_list:

            print(f'\n### {", ".join(case)} --> Features: {feature.upper()}, Model: {model} ###\n')
            
            metrics_df, metrics_summary = train_and_evaluate(endpoint, feature=feature, sources_list=case, model_type=model, 
                                                             save_results=True, experiment_name=f'{"_".join(case)}')
            print(metrics_summary)